<a href="https://colab.research.google.com/github/calebahrens/chessengine/blob/main/Chess_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import random
import sys
king_move = '0'
rook_move = '0'
king_move_player = '0'
rook_move_player = '0'

def score(board):
  score = 0
  piece_values = { 'p': 2, 'P': -2, 'n': 6, 'N': -6, 'b': 6, 'B': -6, 'r': 10, 'R': -10, 'q': 18, 'Q': -18, 'k': 1000, 'K': -1000 }
  for row in range(8):
      for col in range(8):
          piece = board[row][col]
          if piece in piece_values:
              score += piece_values[piece]
          if (row, col) in [(4, 3), (4, 4), (3, 3), (3, 4)]:
              if piece in {'n', 'b', 'r', 'q'}:
                  score += 0.3
              elif piece in {'N', 'B', 'R', 'Q'}:
                  score -= 0.3
              if piece == 'p':
                  score += 0.6
              elif piece == 'P':
                  score -= 0.6
          if (col) == 0 or (col) == 7:
            if piece == 'n':
                score -= 0.2
            if piece == 'N':
                score += 0.2
          if (row) == 0 or (row) == 7:
            if piece == 'n':
                score -= 0.2
            if piece == 'N':
                score += 0.2
          if (row, col) == (4, 4):
            if piece == 'p':
                score += 0.01
  starting_penalty_positions = {
      (7, 1): -0.2, (7, 6): -0.2,
      (7, 2): -0.2, (7, 5): -0.2,
      (0, 1): 0.2, (0, 6): 0.2,
      (0, 2): 0.2, (0, 5): 0.2
  }

  for (row, col), adjustment in starting_penalty_positions.items():
      piece = board[row][col]
      if piece.lower() in {'n', 'b', 'N', 'B'}:
          score += adjustment if piece.islower() else -adjustment

  return score
def pos_to_indices(pos):
  col = ord(pos[0]) - ord('a')
  row = 8 - int(pos[1])
  return row, col

def indices_to_pos(row, col):
  col_pos = chr(col + ord('a'))
  row_pos = str(8 - row)
  return col_pos + row_pos

def indices_to_pos_col(col):
  col_pos = chr(col + ord('a'))
  return col_pos

def indices_to_pos_row(row):
  row_pos = str(8 - row)
  return row_pos

def find_king(board, king_color):
  king_symbol = 'K' if king_color == 'w' else 'k'
  for row in range(8):
    for col in range(8):
      if board[row][col] == king_symbol:
        return row, col
  print('here')
  print_board(board)
  return None

def is_king_in_check(board, king_row, king_col, player):
  opponent = 'b' if player == 'w' else 'w'
  for row in range(8):
      for col in range(8):
          piece = board[row][col]
          if piece != '0' and piece.lower() != 'k' and \
              ((piece.islower() and player == 'w') or (piece.isupper() and player == 'b')):

              if piece.lower() == 'b' or piece.lower() == 'q':
                  directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                  for direction in directions:
                      for i in range(1, 8):
                          new_row = row + i * direction[0]
                          new_col = col + i * direction[1]
                          if 0 <= new_row < 8 and 0 <= new_col < 8:
                              if new_row == king_row and new_col == king_col:
                                  return True
                              if board[new_row][new_col] != '0':
                                  break
                          else:
                              break

              if piece.lower() == 'r' or piece.lower() == 'q':
                  directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                  for direction in directions:
                      for i in range(1, 8):
                          new_row = row + i * direction[0]
                          new_col = col + i * direction[1]
                          if 0 <= new_row < 8 and 0 <= new_col < 8:
                              if new_row == king_row and new_col == king_col:
                                  return True
                              if board[new_row][new_col] != '0':
                                  break
                          else:
                              break

              if piece.lower() == 'n':
                  knight_moves = [(1, 2), (1, -2), (-1, 2), (-1, -2),
                                  (2, 1), (2, -1), (-2, 1), (-2, -1)]
                  for move in knight_moves:
                      new_row = row + move[0]
                      new_col = col + move[1]
                      if 0 <= new_row < 8 and 0 <= new_col < 8:
                          if new_row == king_row and new_col == king_col:
                              return True

              if piece.lower() == 'p':
                  if player == 'w':
                      pawn_attacks = [(1, 1), (1, -1)]
                  else:
                      pawn_attacks = [(-1, 1), (-1, -1)]

                  for attack in pawn_attacks:
                      new_row = row + attack[0]
                      new_col = col + attack[1]
                      if 0 <= new_row < 8 and 0 <= new_col < 8:
                          if new_row == king_row and new_col == king_col:
                              return True

  return False

def players_turn(board):
  from_square = input("From what square: ").strip().lower()
  to_square = input("To what square: ").strip().lower()
  from_row, from_col = pos_to_indices(from_square)
  to_row, to_col = pos_to_indices(to_square)
  s = board[to_row][to_col]
  if board[from_row][from_col] != '0' and s != 'P' and s != 'R' and s != 'N' and s != 'B' and s != 'K' and s != 'Q':
    piece = board[from_row][from_col]
    board[from_row][from_col] = '0'
    board[to_row][to_col] = piece
    print_board(board)
    print()
  else:
    print("Invalid move. Try again.")
    players_turn(board)

def initialize_board():
  board = [[' ' for _ in range(8)] for _ in range(8)]
  board[0] = ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
  board[1] = ['P'] * 8
  board[2] = ['0'] * 8
  board[3] = ['0'] * 8
  board[4] = ['0'] * 8
  board[5] = ['0'] * 8
  board[6] = ['p'] * 8
  board[7] = ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']
  return board

board = initialize_board()

def print_board(board):
    for row in board:
      print(' '.join(str(cell) for cell in row))

def best_move(board):
    global king_move
    global rook_move
    black_king_row, black_king_col = find_king(board, 'b')
    checkmate = False
    checkmate2 = False
    bad_checkmate = False
    previous_score = -6000
    best_moves = []
    rows = list(range(8))
    random.shuffle(rows)
    cols = list(range(8))
    random.shuffle(cols)
    for row in rows:
        for col in cols:
            piece = board[row][col]
            if piece == 'p':
                directions = [1, 2, 3, 4]
                random.shuffle(directions)
                for direction in directions:
                    if direction == 1 and row == 6 and board[row-2][col] == '0' and board[row-1][col] == '0':
                        print(indices_to_pos(row-2, col))
                        board[row][col] = '0'
                        board[row-2][col] = 'p'
                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                            move = best_move_player(board)
                            if move == '0-0':
                              print('0-0')
                              board[0][4] = '0'
                              board[0][6] = 'K'
                              board[0][7] = '0'
                              board[0][5] = 'R'
                            elif move == '0-0-0':
                              print('0-0-0')
                              board[0][4] = '0'
                              board[0][2] = 'K'
                              board[0][0] = '0'
                              board[0][3] = 'R'
                            else:
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                    checkmate = True
                                if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                    bad_checkmate = True
                                if not checkmate and not bad_checkmate:
                                    if best_piece != 'P':
                                      if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                        print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                      else:
                                        print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                    else:
                                        if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                          print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                        else:
                                          print(indices_to_pos(target_row, target_col))
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                            move2 = best_move2(board)
                            if move2 == '0-0':
                              print('0-0')
                              board[0][4] = '0'
                              board[0][6] = 'k'
                              board[0][7] = '0'
                              board[0][5] = 'r'
                              current_score = score(board)
                            elif move2 == '0-0-0':
                              print('0-0-0')
                              board[0][4] = '0'
                              board[0][2] = 'k'
                              board[0][0] = '0'
                              board[0][3] = 'r'
                              current_score = score(board)
                            else:
                                best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                    checkmate2 = True
                                if not checkmate2:
                                    if best_piece2 != 'p':
                                        if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                          print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                        else:
                                          print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                    else:
                                        if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                          print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                        else:
                                          print(indices_to_pos(target_row2, target_col2))
                                    board[best_row2][best_col2] = '0'
                                    board[target_row2][target_col2] = best_piece2
                                    current_score = score(board)
                                    board[best_row2][best_col2] = best_piece2
                                    board[target_row2][target_col2] = captured2
                            if move2 == '0-0':
                              board[0][4] = 'k'
                              board[0][6] = '0'
                              board[0][7] = 'r'
                              board[0][5] = '0'
                            elif move2 == '0-0-0':
                              board[0][4] = 'k'
                              board[0][2] = '0'
                              board[0][0] = 'r'
                              board[0][3] = '0'
                            print(current_score)
                            print()
                            print_board(board)
                            print()
                            if move == '0-0':
                              board[0][4] = 'k'
                              board[0][6] = '0'
                              board[0][7] = 'r'
                              board[0][5] = '0'
                              current_score = score(board)
                            elif move == '0-0-0':
                              board[0][4] = 'k'
                              board[0][2] = '0'
                              board[0][0] = 'r'
                              board[0][3] = '0'
                              current_score = score(board)
                            else:
                                board[best_row][best_col] = best_piece
                                board[target_row][target_col] = captured
                            if current_score > previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row-2, col, 'p')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row-2, col, 'p'))
                        if checkmate:
                            print_board(board)
                            print('CHECKMATE! you lose')
                        board[row][col] = 'p'
                        board[row-2][col] = '0'
                        checkmate = False
                        checkmate2 = False

                    elif direction == 2 and row > 0 and board[row-1][col] == '0':
                        print(indices_to_pos(row-1, col))
                        board[row][col] = '0'
                        board[row-1][col] = 'p'
                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                            move = best_move_player(board)
                            if move == '0-0':
                              print('0-0')
                              board[0][4] = '0'
                              board[0][6] = 'K'
                              board[0][7] = '0'
                              board[0][5] = 'R'
                            elif move == '0-0-0':
                              print('0-0-0')
                              board[0][4] = '0'
                              board[0][2] = 'K'
                              board[0][0] = '0'
                              board[0][3] = 'R'
                            else:
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                  checkmate = True
                                if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                  bad_checkmate = True
                                if not checkmate and not bad_checkmate:
                                    if best_piece != 'P':
                                      if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                        print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                      else:
                                        print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                    else:
                                        if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                          print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                        else:
                                          print(indices_to_pos(target_row, target_col))
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                            move2 = best_move2(board)
                            if move2 == '0-0':
                              print('0-0')
                              board[0][4] = '0'
                              board[0][6] = 'k'
                              board[0][7] = '0'
                              board[0][5] = 'r'
                              current_score = score(board)
                            elif move2 == '0-0-0':
                              print('0-0-0')
                              board[0][4] = '0'
                              board[0][2] = 'k'
                              board[0][0] = '0'
                              board[0][3] = 'r'
                              current_score = score(board)
                            else:
                                best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                    checkmate2 = True
                                if not checkmate2:
                                    if best_piece2 != 'p':
                                        if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                          print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                        else:
                                          print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                    else:
                                        if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                          print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                        else:
                                          print(indices_to_pos(target_row2, target_col2))
                                    board[best_row2][best_col2] = '0'
                                    board[target_row2][target_col2] = best_piece2
                                    current_score = score(board)
                                    board[best_row2][best_col2] = best_piece2
                                    board[target_row2][target_col2] = captured2
                            if move2 == '0-0':
                              board[0][4] = 'k'
                              board[0][6] = '0'
                              board[0][7] = 'r'
                              board[0][5] = '0'
                            elif move2 == '0-0-0':
                              board[0][4] = 'k'
                              board[0][2] = '0'
                              board[0][0] = 'r'
                              board[0][3] = '0'
                            print(current_score)
                            print()
                            print_board(board)
                            print()
                            if move == '0-0':
                              board[0][4] = 'K'
                              board[0][6] = '0'
                              board[0][7] = 'R'
                              board[0][5] = '0'
                            elif move == '0-0-0':
                              board[0][4] = 'K'
                              board[0][2] = '0'
                              board[0][0] = 'R'
                              board[0][3] = '0'
                            else:
                                board[best_row][best_col] = best_piece
                                board[target_row][target_col] = captured
                            if current_score > previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row-1, col, 'p')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row-1, col, 'p'))
                        if checkmate:
                              print_board(board)
                              print('CHECKMATE! you lose')
                        board[row][col] = 'p'
                        board[row-1][col] = '0'
                        checkmate = False
                        checkmate2 = False



                    elif direction == 3 and row > 0 and col > 0 and board[row-1][col-1] in {'P', 'N', 'B', 'R', 'Q'}:
                        captured_piece = board[row-1][col-1]
                        print(col, 'x', indices_to_pos(row-1, col-1))
                        board[row][col] = '0'
                        board[row-1][col-1] = 'p'
                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                            move = best_move_player(board)
                            if move == '0-0':
                              print('0-0')
                              board[0][4] = '0'
                              board[0][6] = 'K'
                              board[0][7] = '0'
                              board[0][5] = 'R'
                            elif move == '0-0-0':
                              print('0-0-0')
                              board[0][4] = '0'
                              board[0][2] = 'K'
                              board[0][0] = '0'
                              board[0][3] = 'R'
                            else:
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                  checkmate = True
                                if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                    bad_checkmate = True
                                if not checkmate and not bad_checkmate:
                                    if best_piece != 'P':
                                      if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                        print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                      else:
                                        print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                    else:
                                        if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                          print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                        else:
                                          print(indices_to_pos(target_row, target_col))
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                            move2 = best_move2(board)
                            if move2 == '0-0':
                              print('0-0')
                              board[0][4] = '0'
                              board[0][6] = 'k'
                              board[0][7] = '0'
                              board[0][5] = 'r'
                              current_score = score(board)
                            elif move2 == '0-0-0':
                              print('0-0-0')
                              board[0][4] = '0'
                              board[0][2] = 'k'
                              board[0][0] = '0'
                              board[0][3] = 'r'
                              current_score = score(board)
                            else:
                                best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                    checkmate2 = True
                                if not checkmate2:
                                    if best_piece2 != 'p':
                                        if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                          print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                        else:
                                          print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                    else:
                                        if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                          print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                        else:
                                          print(indices_to_pos(target_row2, target_col2))
                                    board[best_row2][best_col2] = '0'
                                    board[target_row2][target_col2] = best_piece2
                                    current_score = score(board)
                                    board[best_row2][best_col2] = best_piece2
                                    board[target_row2][target_col2] = captured2
                            if move2 == '0-0':
                              board[0][4] = 'k'
                              board[0][6] = '0'
                              board[0][7] = 'r'
                              board[0][5] = '0'
                            elif move2 == '0-0-0':
                              board[0][4] = 'k'
                              board[0][2] = '0'
                              board[0][0] = 'r'
                              board[0][3] = '0'
                            print(current_score)
                            print()
                            print_board(board)
                            print()
                            if move == '0-0':
                              board[0][4] = 'K'
                              board[0][6] = '0'
                              board[0][7] = 'R'
                              board[0][5] = '0'
                            elif move == '0-0-0':
                              board[0][4] = 'K'
                              board[0][2] = '0'
                              board[0][0] = 'R'
                              board[0][3] = '0'
                            else:
                                board[best_row][best_col] = best_piece
                                board[target_row][target_col] = captured
                            if current_score > previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row-1, col-1, 'p')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row-1, col-1, 'p'))
                        if checkmate:
                            print_board(board)
                            print('CHECKMATE! you lose')
                        board[row][col] = 'p'
                        board[row-1][col-1] = captured_piece
                        checkmate = False
                        checkmate2 = False

                    elif direction == 4 and row > 0 and col < 7 and board[row-1][col+1] in {'P', 'N', 'B', 'R', 'Q'}:
                        captured_piece = board[row-1][col+1]
                        print(col, 'x', indices_to_pos(row-1, col+1))
                        board[row][col] = '0'
                        board[row-1][col+1] = 'p'
                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                            move = best_move_player(board)
                            if move == '0-0':
                              print('0-0')
                              board[0][4] = '0'
                              board[0][6] = 'K'
                              board[0][7] = '0'
                              board[0][5] = 'R'
                            elif move == '0-0-0':
                              print('0-0-0')
                              board[0][4] = '0'
                              board[0][2] = 'K'
                              board[0][0] = '0'
                              board[0][3] = 'R'
                            else:
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                  checkmate = True
                                if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                    bad_checkmate = True
                                if not checkmate and not bad_checkmate:
                                    if best_piece != 'P':
                                      if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                        print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                      else:
                                        print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                    else:
                                        if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                          print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                        else:
                                          print(indices_to_pos(target_row, target_col))
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                            move2 = best_move2(board)
                            if move2 == '0-0':
                              print('0-0')
                              board[0][4] = '0'
                              board[0][6] = 'k'
                              board[0][7] = '0'
                              board[0][5] = 'r'
                              current_score = score(board)
                            elif move2 == '0-0-0':
                              print('0-0-0')
                              board[0][4] = '0'
                              board[0][2] = 'k'
                              board[0][0] = '0'
                              board[0][3] = 'r'
                              current_score = score(board)
                            else:
                                best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                    checkmate2 = True
                                if not checkmate2:
                                    if best_piece2 != 'p':
                                        if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                          print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                        else:
                                          print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                    else:
                                        if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                          print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                        else:
                                          print(indices_to_pos(target_row2, target_col2))
                                    board[best_row2][best_col2] = '0'
                                    board[target_row2][target_col2] = best_piece2
                                    current_score = score(board)
                            if move2 == '0-0':
                              board[0][4] = 'k'
                              board[0][6] = '0'
                              board[0][7] = 'r'
                              board[0][5] = '0'
                            elif move2 == '0-0-0':
                              board[0][4] = 'k'
                              board[0][2] = '0'
                              board[0][0] = 'r'
                              board[0][3] = '0'
                            print(current_score)
                            print()
                            print_board(board)
                            print()
                            board[best_row2][best_col2] = best_piece2
                            board[target_row2][target_col2] = captured2
                            if move == '0-0':
                              board[0][4] = 'K'
                              board[0][6] = '0'
                              board[0][7] = 'R'
                              board[0][5] = '0'
                            elif move == '0-0-0':
                              board[0][4] = 'K'
                              board[0][2] = '0'
                              board[0][0] = 'R'
                              board[0][3] = '0'
                            else:
                                board[best_row][best_col] = best_piece
                                board[target_row][target_col] = captured
                            if current_score > previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row-1, col+1, 'p')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row-1, col+1, 'p'))
                        if checkmate:
                            print_board(board)
                            print('CHECKMATE! you lose')
                        board[row][col] = 'p'
                        board[row-1][col+1] = captured_piece
                        checkmate = False
                        checkmate2 = False


            elif piece == 'n':
                directions = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]
                random.shuffle(directions)
                for direction in directions:
                    new_row = row + direction[0]
                    new_col = col + direction[1]
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                            captured_piece = board[new_row][new_col]
                            if board[new_row][new_col] in {'P', 'N', 'B', 'R', 'Q'}:
                              print('N' + 'x' + indices_to_pos(new_row, new_col))
                            else:
                              print('N' + indices_to_pos(new_row, new_col))
                            board[row][col] = '0'
                            board[new_row][new_col] = 'n'
                            if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                move = best_move_player(board)
                                if move == '0-0':
                                  print('0-0')
                                  board[0][4] = '0'
                                  board[0][6] = 'K'
                                  board[0][7] = '0'
                                  board[0][5] = 'R'
                                elif move == '0-0-0':
                                  print('0-0-0')
                                  board[0][4] = '0'
                                  board[0][2] = 'K'
                                  board[0][0] = '0'
                                  board[0][3] = 'R'
                                else:
                                    best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                    print(best_row, best_col, target_row, target_col, best_piece, captured)
                                    if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                        print('TRUE!')
                                        checkmate = True
                                    if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                        bad_checkmate = True
                                    if not checkmate and not bad_checkmate:
                                        if best_piece != 'P':
                                          if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                            print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                          else:
                                            print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                        else:
                                          if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                            print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                          else:
                                            print(indices_to_pos(target_row, target_col))
                                        board[best_row][best_col] = '0'
                                        board[target_row][target_col] = best_piece
                                move2 = best_move2(board)
                                if move2 == '0-0':
                                  print('0-0')
                                  board[0][4] = '0'
                                  board[0][6] = 'k'
                                  board[0][7] = '0'
                                  board[0][5] = 'r'
                                  current_score = score(board)
                                elif move2 == '0-0-0':
                                  print('0-0-0')
                                  board[0][4] = '0'
                                  board[0][2] = 'k'
                                  board[0][0] = '0'
                                  board[0][3] = 'r'
                                  current_score = score(board)
                                else:
                                    best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                    if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                        checkmate2 = True
                                    if not checkmate2:
                                        if best_piece2 != 'p':
                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                              print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                            else:
                                              print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                        else:
                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                              print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                            else:
                                              print(indices_to_pos(target_row2, target_col2))
                                        board[best_row2][best_col2] = '0'
                                        board[target_row2][target_col2] = best_piece2
                                        current_score = score(board)
                                print(current_score)
                                print()
                                print_board(board)
                                print()
                                board[best_row2][best_col2] = best_piece2
                                board[target_row2][target_col2] = captured2
                                if move == '0-0':
                                  board[0][4] = 'K'
                                  board[0][6] = '0'
                                  board[0][7] = 'R'
                                  board[0][5] = '0'
                                elif move == '0-0-0':
                                  board[0][4] = 'K'
                                  board[0][2] = '0'
                                  board[0][0] = 'R'
                                  board[0][3] = '0'
                                else:
                                    board[best_row][best_col] = best_piece
                                    board[target_row][target_col] = captured
                                if current_score > previous_score:
                                    previous_score = current_score
                                    best_moves = [(row, col, new_row, new_col, 'n')]
                                elif current_score == previous_score:
                                    best_moves.append((row, col, new_row, new_col, 'n'))
                            if checkmate:
                                print_board(board)
                                print('CHECKMATE! you lose')
                            board[row][col] = 'n'
                            board[new_row][new_col] = captured_piece
                            checkmate = False
                            checkmate2 = False


            elif piece == 'b':
                directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                captured_piece = board[new_row][new_col]
                                if board[new_row][new_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                  print('B' + 'x' + indices_to_pos(new_row, new_col))
                                else:
                                  print('B' + indices_to_pos(new_row, new_col))
                                board[row][col] = '0'
                                board[new_row][new_col] = 'b'
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    move = best_move_player(board)
                                    if move == '0-0':
                                      print('0-0')
                                      board[0][4] = '0'
                                      board[0][6] = 'K'
                                      board[0][7] = '0'
                                      board[0][5] = 'R'
                                    elif move == '0-0-0':
                                      print('0-0-0')
                                      board[0][4] = '0'
                                      board[0][2] = 'K'
                                      board[0][0] = '0'
                                      board[0][3] = 'R'
                                    else:
                                        best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                        if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                            checkmate = True
                                        if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                            bad_checkmate = True
                                        if not checkmate and not bad_checkmate:
                                            if best_piece != 'P':
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                            else:
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(indices_to_pos(target_row, target_col))
                                            board[best_row][best_col] = '0'
                                            board[target_row][target_col] = best_piece
                                    move2 = best_move2(board)
                                    if move2 == '0-0':
                                      print('0-0')
                                      board[0][4] = '0'
                                      board[0][6] = 'k'
                                      board[0][7] = '0'
                                      board[0][5] = 'r'
                                      current_score = score(board)
                                    elif move2 == '0-0-0':
                                      print('0-0-0')
                                      board[0][4] = '0'
                                      board[0][2] = 'k'
                                      board[0][0] = '0'
                                      board[0][3] = 'r'
                                      current_score = score(board)
                                    else:
                                        best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                        if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                            checkmate2 = True
                                        if not checkmate2:
                                            if best_piece2 != 'p':
                                              if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                              else:
                                                print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                            else:
                                              if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                              else:
                                                print(indices_to_pos(target_row2, target_col2))
                                            board[best_row2][best_col2] = '0'
                                            board[target_row2][target_col2] = best_piece2
                                            current_score = score(board)
                                    if move2 == '0-0':
                                      board[0][4] = 'k'
                                      board[0][6] = '0'
                                      board[0][7] = 'r'
                                      board[0][5] = '0'
                                    elif move2 == '0-0-0':
                                      board[0][4] = 'k'
                                      board[0][2] = '0'
                                      board[0][0] = 'r'
                                      board[0][3] = '0'
                                    print(current_score)
                                    print()
                                    print_board(board)
                                    print()
                                    board[best_row2][best_col2] = best_piece2
                                    board[target_row2][target_col2] = captured2
                                    if move == '0-0':
                                      board[0][4] = 'K'
                                      board[0][6] = '0'
                                      board[0][7] = 'R'
                                      board[0][5] = '0'
                                    elif move == '0-0-0':
                                      board[0][4] = 'K'
                                      board[0][2] = '0'
                                      board[0][0] = 'R'
                                      board[0][3] = '0'
                                    else:
                                        board[best_row][best_col] = best_piece
                                        board[target_row][target_col] = captured
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'b')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'b'))
                                if checkmate:
                                    print_board(board)
                                    print('CHECKMATE! you lose')
                                board[row][col] = 'b'
                                board[new_row][new_col] = captured_piece
                                checkmate = False
                                checkmate2 = False

                            else:
                                break
                            if board[new_row][new_col] == 'P' or board[new_row][new_col] == 'N' or board[new_row][new_col] == 'B' or board[new_row][new_col] == 'R' or board[new_row][new_col] == 'Q':
                                break
                        else:
                            break

            elif piece == 'r':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                captured_piece = board[new_row][new_col]
                                if board[new_row][new_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                  print('R' + 'x' + indices_to_pos(new_row, new_col))
                                else:
                                  print('R' + indices_to_pos(new_row, new_col))
                                board[row][col] = '0'
                                board[new_row][new_col] = 'r'
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    move = best_move_player(board)
                                    if move == '0-0':
                                      print('0-0')
                                      board[0][4] = '0'
                                      board[0][6] = 'K'
                                      board[0][7] = '0'
                                      board[0][5] = 'R'
                                    elif move == '0-0-0':
                                      print('0-0-0')
                                      board[0][4] = '0'
                                      board[0][2] = 'K'
                                      board[0][0] = '0'
                                      board[0][3] = 'R'
                                    else:
                                        best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                        if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                            checkmate = True
                                        if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                            bad_checkmate = True
                                        if not checkmate and not bad_checkmate:
                                            if best_piece != 'P':
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                            else:
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(indices_to_pos(target_row, target_col))
                                            board[best_row][best_col] = '0'
                                            board[target_row][target_col] = best_piece
                                    move2 = best_move2(board)
                                    if move2 == '0-0':
                                      print('0-0')
                                      board[0][4] = '0'
                                      board[0][6] = 'k'
                                      board[0][7] = '0'
                                      board[0][5] = 'r'
                                      current_score = score(board)
                                    elif move2 == '0-0-0':
                                      print('0-0-0')
                                      board[0][4] = '0'
                                      board[0][2] = 'k'
                                      board[0][0] = '0'
                                      board[0][3] = 'r'
                                      current_score = score(board)
                                    else:
                                        best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                        if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                            checkmate2 = True
                                        if not checkmate2:
                                            if best_piece2 != 'p':
                                              if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                              else:
                                                print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                            else:
                                              if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                              else:
                                                print(indices_to_pos(target_row2, target_col2))
                                            board[best_row2][best_col2] = '0'
                                            board[target_row2][target_col2] = best_piece2
                                            current_score = score(board)
                                    if move2 == '0-0':
                                      board[0][4] = 'k'
                                      board[0][6] = '0'
                                      board[0][7] = 'r'
                                      board[0][5] = '0'
                                    elif move2 == '0-0-0':
                                      board[0][4] = 'k'
                                      board[0][2] = '0'
                                      board[0][0] = 'r'
                                      board[0][3] = '0'
                                    print(current_score)
                                    print()
                                    print_board(board)
                                    print()
                                    board[best_row2][best_col2] = best_piece2
                                    board[target_row2][target_col2] = captured2
                                    if move == '0-0':
                                      board[0][4] = 'K'
                                      board[0][6] = '0'
                                      board[0][7] = 'R'
                                      board[0][5] = '0'
                                    elif move == '0-0-0':
                                      board[0][4] = 'K'
                                      board[0][2] = '0'
                                      board[0][0] = 'R'
                                      board[0][3] = '0'
                                    else:
                                        board[best_row][best_col] = best_piece
                                        board[target_row][target_col] = captured
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'r')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'r'))
                                if checkmate:
                                    print_board(board)
                                    print('CHECKMATE! you lose')
                                board[row][col] = 'r'
                                board[new_row][new_col] = captured_piece
                                checkmate = False
                                checkmate2 = False

                            else:
                                break
                            if board[new_row][new_col] == 'P' or board[new_row][new_col] == 'N' or board[new_row][new_col] == 'B' or board[new_row][new_col] == 'R' or board[new_row][new_col] == 'Q':
                                break
                        else:
                            break

            elif piece == 'q':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                captured_piece = board[new_row][new_col]
                                if board[new_row][new_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                  print('Q' + 'x' + indices_to_pos(new_row, new_col))
                                else:
                                  print('Q' + indices_to_pos(new_row, new_col))
                                board[row][col] = '0'
                                board[new_row][new_col] = 'q'
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    move = best_move_player(board)
                                    if move == '0-0':
                                      print('0-0')
                                      board[0][4] = '0'
                                      board[0][6] = 'K'
                                      board[0][7] = '0'
                                      board[0][5] = 'R'
                                    elif move == '0-0-0':
                                      print('0-0-0')
                                      board[0][4] = '0'
                                      board[0][2] = 'K'
                                      board[0][0] = '0'
                                      board[0][3] = 'R'
                                    else:
                                        best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                        if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                            checkmate = True
                                            print('checkmate = True')
                                        if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                            bad_checkmate = True
                                            print('bad_checkmate = True')
                                        print('hello')
                                        if not checkmate and not bad_checkmate:
                                            print('hi')
                                            if best_piece != 'P':
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                            else:
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(indices_to_pos(target_row, target_col))
                                            board[best_row][best_col] = '0'
                                            board[target_row][target_col] = best_piece
                                    move2 = best_move2(board)
                                    if move2 == '0-0':
                                      print('0-0')
                                      board[0][4] = '0'
                                      board[0][6] = 'k'
                                      board[0][7] = '0'
                                      board[0][5] = 'r'
                                      current_score = score(board)
                                    elif move2 == '0-0-0':
                                      print('0-0-0')
                                      board[0][4] = '0'
                                      board[0][2] = 'k'
                                      board[0][0] = '0'
                                      board[0][3] = 'r'
                                      current_score = score(board)
                                    else:
                                        best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                        if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                            checkmate2 = True
                                        if not checkmate2:
                                            if best_piece2 != 'p':
                                              if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                              else:
                                                print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                            else:
                                              if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                              else:
                                                print(indices_to_pos(target_row2, target_col2))
                                            board[best_row2][best_col2] = '0'
                                            board[target_row2][target_col2] = best_piece2
                                            current_score = score(board)
                                    if move2 == '0-0':
                                      board[0][4] = 'k'
                                      board[0][6] = '0'
                                      board[0][7] = 'r'
                                      board[0][5] = '0'
                                    elif move2 == '0-0-0':
                                      board[0][4] = 'k'
                                      board[0][2] = '0'
                                      board[0][0] = 'r'
                                      board[0][3] = '0'
                                    print(current_score)
                                    print()
                                    print_board(board)
                                    print()
                                    board[best_row2][best_col2] = best_piece2
                                    board[target_row2][target_col2] = captured2
                                    if move == '0-0':
                                      board[0][4] = 'K'
                                      board[0][6] = '0'
                                      board[0][7] = 'R'
                                      board[0][5] = '0'
                                    elif move == '0-0-0':
                                      board[0][4] = 'K'
                                      board[0][2] = '0'
                                      board[0][0] = 'R'
                                      board[0][3] = '0'
                                    else:
                                        board[best_row][best_col] = best_piece
                                        board[target_row][target_col] = captured
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'q')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'q'))
                                if checkmate:
                                    print_board(board)
                                    print('CHECKMATE! you lose')
                                board[row][col] = 'q'
                                board[new_row][new_col] = captured_piece
                                checkmate = False
                                checkmate2 = False

                            else:
                                break
                            if board[new_row][new_col] == 'P' or board[new_row][new_col] == 'N' or board[new_row][new_col] == 'B' or board[new_row][new_col] == 'R' or board[new_row][new_col] == 'Q':
                                break
                        else:
                            break

            elif piece == 'k':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1), '0-0', '0-0-0']
                random.shuffle(directions)
                for direction in directions:
                  if direction in {(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)}:
                    new_row = row + direction[0]
                    new_col = col + direction[1]
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                            captured_piece = board[new_row][new_col]
                            if board[new_row][new_col] in {'P', 'N', 'B', 'R', 'Q'}:
                              print('K' + 'x' + indices_to_pos(new_row, new_col))
                            else:
                              print('K' + indices_to_pos(new_row, new_col))
                            board[row][col] = '0'
                            board[new_row][new_col] = 'k'
                            black_king_row, black_king_col = find_king(board, 'b')
                            if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                move = best_move_player(board)
                                if move == '0-0':
                                  print('0-0')
                                  board[0][4] = '0'
                                  board[0][6] = 'K'
                                  board[0][7] = '0'
                                  board[0][5] = 'R'
                                elif move == '0-0-0':
                                  print('0-0-0')
                                  board[0][4] = '0'
                                  board[0][2] = 'K'
                                  board[0][0] = '0'
                                  board[0][3] = 'R'
                                else:
                                    best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                    if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                        checkmate = True
                                    if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                        bad_checkmate = True
                                    if not checkmate and not bad_checkmate:
                                        if best_piece != 'P':
                                          if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                            print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                          else:
                                            print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                        else:
                                            if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                              print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                            else:
                                              print(indices_to_pos(target_row, target_col))
                                        board[best_row][best_col] = '0'
                                        board[target_row][target_col] = best_piece
                                move2 = best_move2(board)
                                if move2 == '0-0':
                                  print('0-0')
                                  board[0][4] = '0'
                                  board[0][6] = 'k'
                                  board[0][7] = '0'
                                  board[0][5] = 'r'
                                  current_score = score(board)
                                elif move2 == '0-0-0':
                                  print('0-0-0')
                                  board[0][4] = '0'
                                  board[0][2] = 'k'
                                  board[0][0] = '0'
                                  board[0][3] = 'r'
                                  current_score = score(board)
                                else:
                                    best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                    if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                        checkmate2 = True
                                    if not checkmate2:
                                        if best_piece2 != 'p':
                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                              print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                            else:
                                              print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                        else:
                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                              print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                            else:
                                              print(indices_to_pos(target_row2, target_col2))
                                        board[best_row2][best_col2] = '0'
                                        board[target_row2][target_col2] = best_piece2
                                        current_score = score(board)
                                if move2 == '0-0':
                                  board[0][4] = 'k'
                                  board[0][6] = '0'
                                  board[0][7] = 'r'
                                  board[0][5] = '0'
                                elif move2 == '0-0-0':
                                  board[0][4] = 'k'
                                  board[0][2] = '0'
                                  board[0][0] = 'r'
                                  board[0][3] = '0'
                                print(current_score)
                                print()
                                print_board(board)
                                print()
                                board[best_row2][best_col2] = best_piece2
                                board[target_row2][target_col2] = captured2
                                if move == '0-0':
                                  board[0][4] = 'K'
                                  board[0][6] = '0'
                                  board[0][7] = 'R'
                                  board[0][5] = '0'
                                elif move == '0-0-0':
                                  board[0][4] = 'K'
                                  board[0][2] = '0'
                                  board[0][0] = 'R'
                                  board[0][3] = '0'
                                else:
                                    board[best_row][best_col] = best_piece
                                    board[target_row][target_col] = captured
                                if current_score > previous_score:
                                    previous_score = current_score
                                    best_moves = [(row, col, new_row, new_col, 'k')]
                                elif current_score == previous_score:
                                    best_moves.append((row, col, new_row, new_col, 'k'))
                            if checkmate:
                                print_board(board)
                                print('CHECKMATE! you lose')
                            else:
                                print('In check')
                            board[row][col] = 'k'
                            board[new_row][new_col] = captured_piece
                            checkmate = False
                            checkmate2 = False
                            black_king_row, black_king_col = find_king(board, 'b')
                  else:
                      if direction == '0-0':
                          if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                              if board[7][4] == 'k' and board[7][7] == 'r' and rook_move == '0' and king_move == '0' and board[7][5] == '0' and board[7][6] == '0':
                                  board[7][5] = 'k'
                                  board[7][4] = '0'
                                  if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                      board[7][6] = 'k'
                                      board[7][5] = '0'
                                      if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                          board[7][7] = '0'
                                          board[7][5] = 'r'
                                          best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                          if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                              checkmate = True
                                          if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                              bad_checkmate = True
                                          if not checkmate and not bad_checkmate:
                                              if best_piece != 'P':
                                                if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                  print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                                else:
                                                  print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                              else:
                                                  if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                    print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                                  else:
                                                    print(indices_to_pos(target_row, target_col))
                                              board[best_row][best_col] = '0'
                                              board[target_row][target_col] = best_piece
                                              best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                              if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                                  checkmate2 = True
                                              if not checkmate2:
                                                  if best_piece2 != 'p':
                                                      if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                        print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                                      else:
                                                        print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                                  else:
                                                      if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                        print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                                      else:
                                                        print(indices_to_pos(target_row2, target_col2))
                                                  board[best_row2][best_col2] = '0'
                                                  board[target_row2][target_col2] = best_piece2
                                                  current_score = score(board)
                                                  print(current_score)
                                                  print()
                                                  print_board(board)
                                                  print()
                                                  board[best_row2][best_col2] = best_piece2
                                                  board[target_row2][target_col2] = captured2
                                                  board[best_row][best_col] = best_piece
                                                  board[target_row][target_col] = captured
                                                  if current_score > previous_score:
                                                      previous_score = current_score
                                                      best_moves = [('0-0')]
                                                  elif current_score == previous_score:
                                                      best_moves.append(('0-0'))
                                          elif checkmate:
                                                  print_board(board)
                                                  print('CHECKMATE! you lose')
                                      else:
                                          print('In check')
                                      board[7][4] = 'k'
                                      board[7][6] = '0'
                                      board[7][7] = 'r'
                                      board[7][5] = '0'
                                      checkmate = False
                                      checkmate2 = False
                                      black_king_row, black_king_col = find_king(board, 'b')
                                  else:
                                      board[7][4] = 'k'
                                      board[7][6] = '0'
                              else:
                                  board[7][4] = 'k'
                                  board[7][5] = '0'
                      elif direction == '0-0-0':
                          if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                              if board[7][4] == 'k' and board[7][0] == 'r' and rook_move == '0' and king_move == '0' and board[7][1] == '0' and board[7][2] == '0' and board[7][3] == '0':
                                  board[7][3] = 'k'
                                  board[7][4] = '0'
                                  if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                      board[7][2] = 'k'
                                      board[7][3] = '0'
                                      if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                          board[7][0] = '0'
                                          board[7][3] = 'r'
                                          best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                          if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                              checkmate = True
                                          if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                              bad_checkmate = True
                                          if not checkmate and not bad_checkmate:
                                              if best_piece != 'P':
                                                if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                  print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                                else:
                                                  print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                              else:
                                                  if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                    print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                                  else:
                                                    print(indices_to_pos(target_row, target_col))
                                              board[best_row][best_col] = '0'
                                              board[target_row][target_col] = best_piece
                                              best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                              if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                                  checkmate2 = True
                                              if not checkmate2:
                                                  if best_piece2 != 'p':
                                                      if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                        print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                                      else:
                                                        print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                                  else:
                                                      if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                        print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                                      else:
                                                        print(indices_to_pos(target_row2, target_col2))
                                                  board[best_row2][best_col2] = '0'
                                                  board[target_row2][target_col2] = best_piece2
                                                  current_score = score(board)
                                                  print(current_score)
                                                  print()
                                                  print_board(board)
                                                  print()
                                                  board[best_row2][best_col2] = best_piece2
                                                  board[target_row2][target_col2] = captured2
                                                  board[best_row][best_col] = best_piece
                                                  board[target_row][target_col] = captured
                                                  if current_score > previous_score:
                                                      previous_score = current_score
                                                      best_moves = [('0-0-0')]
                                                  elif current_score == previous_score:
                                                      best_moves.append(('0-0-0'))
                                                  elif checkmate:
                                                      print_board(board)
                                                      print('CHECKMATE! you lose')
                                              else:
                                                  print('In check')
                                              board[7][4] = 'k'
                                              board[7][2] = '0'
                                              board[7][0] = 'r'
                                              board[7][3] = '0'
                                              checkmate = False
                                              checkmate2 = False
                                              black_king_row, black_king_col = find_king(board, 'b')
                                  else:
                                      board[7][4] = 'k'
                                      board[7][2] = '0'
                              else:
                                  board[7][4] = 'k'
                                  board[7][3] = '0'



    if best_moves:
        best_move = random.choice(best_moves)
        if best_move != '0-0' and best_move != '0-0-0':
            best_row, best_col, target_row, target_col, best_piece = best_move
            captured_piece = board[target_row][target_col]
            piece = board[target_row][target_col]
            board[best_row][best_col] = '0'
            board[target_row][target_col] = best_piece
            if best_piece == 'k':
                king_move = '1'
            print_board(board)
            print()
            if best_piece != 'p':
                  if piece in {'P', 'N', 'B', 'R', 'Q'}:
                    print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                  else:
                    print(best_piece.upper() + indices_to_pos(target_row, target_col))
            else:
                  if piece in {'P', 'N', 'B', 'R', 'Q'}:
                    print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                  else:
                    print(indices_to_pos(target_row, target_col))
        elif best_move == '0-0':
            king_move = '1'
            board[7][4] = '0'
            board[7][7] = '0'
            board[7][6] = 'k'
            board[7][5] = 'r'
            print('0-0')
        elif best_move == '0-0-0':
            king_move = '1'
            board[7][4] = '0'
            board[7][0] = '0'
            board[7][2] = 'k'
            board[7][3] = 'r'
            print('0-0-0')
        print(previous_score)
        print()
        players_turn(board)

def best_move_player(board):
    global rook_move_player
    global king_move_player
    white_king_row, white_king_col = find_king(board, 'w')
    checkmate = False
    previous_score = 60
    best_moves = []
    rows = list(range(8))
    random.shuffle(rows)
    cols = list(range(8))
    random.shuffle(cols)
    for row in rows:
        for col in cols:
            piece = board[row][col]
            if piece == 'P':
                directions = [1, 2, 3, 4]
                random.shuffle(directions)
                for direction in directions:
                    if direction == 1 and row == 1 and board[row+2][col] == '0' and board[row+1][col] == '0':
                        board[row][col] = '0'
                        board[row+2][col] = 'P'
                        if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                            move = best_move2(board)
                            if move == '0-0':
                              board[7][4] = '0'
                              board[7][6] = 'k'
                              board[7][7] = '0'
                              board[7][5] = 'r'
                              current_score = score(board)
                            elif move == '0-0-0':
                              board[7][4] = '0'
                              board[7][0] = '0'
                              board[7][2] = 'k'
                              board[7][3] = 'r'
                              current_score = score(board)
                            else:
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                    checkmate = True
                                    print('hahaha')
                                if not checkmate:
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                                    current_score = score(board)
                                    board[best_row][best_col] = best_piece
                                    board[target_row][target_col] = captured
                            if move == '0-0':
                                board[7][4] = 'k'
                                board[7][6] = '0'
                                board[7][7] = 'r'
                                board[7][5] = '0'
                            elif move == '0-0-0':
                              board[7][4] = 'k'
                              board[7][0] = 'r'
                              board[7][2] = '0'
                              board[7][3] = '0'
                            if current_score < previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row+2, col, 'P')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row+2, col, 'P'))
                                if best_piece != 'p':
                                  if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                    analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                  else:
                                    analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                else:
                                    if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                      analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                    else:
                                      analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                            if checkmate:
                                return 2, 2, 2, 2, 2, 2
                        board[row][col] = 'P'
                        board[row+2][col] = '0'
                        checkmate = False

                    elif direction == 2 and row < 7 and board[row+1][col] == '0':
                        board[row][col] = '0'
                        board[row+1][col] = 'P'
                        if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                            move = best_move2(board)
                            if move == '0-0':
                              board[7][4] = '0'
                              board[7][6] = 'k'
                              board[7][7] = '0'
                              board[7][5] = 'r'
                              current_score = score(board)
                            elif move == '0-0-0':
                              board[7][4] = '0'
                              board[7][0] = '0'
                              board[7][2] = 'k'
                              board[7][3] = 'r'
                              current_score = score(board)
                            else:
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                    checkmate = True
                                    print('hahaha2')
                                if not checkmate:
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                                    current_score = score(board)
                                    board[best_row][best_col] = best_piece
                                    board[target_row][target_col] = captured
                            if move == '0-0':
                              board[7][4] = 'k'
                              board[7][6] = '0'
                              board[7][7] = 'r'
                              board[7][5] = '0'
                            elif move == '0-0-0':
                              board[7][4] = 'k'
                              board[7][0] = 'r'
                              board[7][2] = '0'
                              board[7][3] = '0'
                            if current_score < previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row+1, col, 'P')]
                                if best_piece != 'p':
                                  if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                    analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                  else:
                                    analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                else:
                                    if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                      analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                    else:
                                      analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                            elif current_score == previous_score:
                                best_moves.append((row, col, row+1, col, 'P'))
                            if checkmate:
                                return 2, 2, 2, 2, 2, 2
                        board[row][col] = 'P'
                        board[row+1][col] = '0'
                        checkmate = False

                    elif direction == 3 and row < 7 and col > 0 and board[row+1][col-1] in {'p', 'n', 'b', 'r', 'q'}:
                        captured_piece = board[row+1][col-1]
                        board[row][col] = '0'
                        board[row+1][col-1] = 'P'
                        if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                            move = best_move2(board)
                            if move == '0-0':
                              board[7][4] = '0'
                              board[7][6] = 'k'
                              board[7][7] = '0'
                              board[7][5] = 'r'
                              current_score = score(board)
                            elif move == '0-0-0':
                              board[7][4] = '0'
                              board[7][0] = '0'
                              board[7][2] = 'k'
                              board[7][3] = 'r'
                              current_score = score(board)
                            else:
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                    checkmate = True
                                    print('hahaha3')
                                if not checkmate:
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                                    current_score = score(board)
                                    board[best_row][best_col] = best_piece
                                    board[target_row][target_col] = captured
                            if move == '0-0':
                              board[7][4] = 'k'
                              board[7][6] = '0'
                              board[7][7] = 'r'
                              board[7][5] = '0'
                            elif move == '0-0-0':
                              board[7][4] = 'k'
                              board[7][0] = 'r'
                              board[7][2] = '0'
                              board[7][3] = '0'
                            if current_score < previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row+1, col-1, 'P')]
                                if best_piece != 'p':
                                  if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                    analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                  else:
                                    analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                else:
                                    if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                      analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                    else:
                                      analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                            elif current_score == previous_score:
                                best_moves.append((row, col, row+1, col-1, 'P'))
                            if checkmate:
                                return 2, 2, 2, 2, 2, 2
                        board[row][col] = 'P'
                        board[row+1][col-1] = captured_piece
                        checkmate = False

                    elif direction == 4 and row > 0 and col < 7 and board[row+1][col+1] in {'p', 'n', 'b', 'r', 'q'}:
                        captured_piece = board[row+1][col+1]
                        board[row][col] = '0'
                        board[row+1][col+1] = 'P'
                        if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                            move = best_move2(board)
                            if move == '0-0':
                              board[7][4] = '0'
                              board[7][6] = 'k'
                              board[7][7] = '0'
                              board[7][5] = 'r'
                              current_score = score(board)
                            elif move == '0-0-0':
                              board[7][4] = '0'
                              board[7][0] = '0'
                              board[7][2] = 'k'
                              board[7][3] = 'r'
                              current_score = score(board)
                            else:
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                    checkmate = True
                                    print('hahaha4')
                                if not checkmate:
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                                    current_score = score(board)
                                    board[best_row][best_col] = best_piece
                                    board[target_row][target_col] = captured
                            if move == '0-0':
                              board[7][4] = 'k'
                              board[7][6] = '0'
                              board[7][7] = 'r'
                              board[7][5] = '0'
                            elif move == '0-0-0':
                              board[7][4] = 'k'
                              board[7][0] = 'r'
                              board[7][2] = '0'
                              board[7][3] = '0'
                            if current_score < previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row+1, col+1, 'P')]
                                if best_piece != 'p':
                                  if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                    analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                  else:
                                    analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                else:
                                    if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                      analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                    else:
                                      analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                            elif current_score == previous_score:
                                best_moves.append((row, col, row+1, col+1, 'P'))
                            if checkmate:
                                return 2, 2, 2, 2, 2, 2
                        board[row][col] = 'P'
                        board[row+1][col+1] = captured_piece
                        checkmate = False

            elif piece == 'N':
                directions = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]
                random.shuffle(directions)
                for direction in directions:
                    new_row = row + direction[0]
                    new_col = col + direction[1]
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if board[new_row][new_col] in {'0', 'p', 'n', 'b', 'r', 'q'}:
                            captured_piece = board[new_row][new_col]
                            board[row][col] = '0'
                            board[new_row][new_col] = 'N'
                            if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                move = best_move2(board)
                                if move == '0-0':
                                  board[7][4] = '0'
                                  board[7][6] = 'k'
                                  board[7][7] = '0'
                                  board[7][5] = 'r'
                                  current_score = score(board)
                                elif move == '0-0-0':
                                  board[7][4] = '0'
                                  board[7][0] = '0'
                                  board[7][2] = 'k'
                                  board[7][3] = 'r'
                                  current_score = score(board)
                                else:
                                    best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                    if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                        checkmate = True
                                        print('hahaha N')
                                    if not checkmate:
                                        board[best_row][best_col] = '0'
                                        board[target_row][target_col] = best_piece
                                        current_score = score(board)
                                        board[best_row][best_col] = best_piece
                                        board[target_row][target_col] = captured
                                if move == '0-0':
                                  board[7][4] = 'k'
                                  board[7][6] = '0'
                                  board[7][7] = 'r'
                                  board[7][5] = '0'
                                elif move == '0-0-0':
                                  board[7][4] = 'k'
                                  board[7][0] = 'r'
                                  board[7][2] = '0'
                                  board[7][3] = '0'
                                if current_score < previous_score:
                                    previous_score = current_score
                                    best_moves = [(row, col, new_row, new_col, 'N')]
                                    if best_piece != 'p':
                                      if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                        analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                      else:
                                        analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                    else:
                                        if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                          analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                        else:
                                          analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                elif current_score == previous_score:
                                    best_moves.append((row, col, new_row, new_col, 'N'))
                                if checkmate:
                                    return 2, 2, 2, 2, 2, 2
                            board[row][col] = 'N'
                            board[new_row][new_col] = captured_piece
                            checkmate = False

            elif piece == 'B':
                directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'p', 'n', 'b', 'r', 'q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'B'
                                if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                    move = best_move2(board)
                                    if move == '0-0':
                                      board[7][4] = '0'
                                      board[7][6] = 'k'
                                      board[7][7] = '0'
                                      board[7][5] = 'r'
                                      current_score = score(board)
                                    elif move == '0-0-0':
                                      board[7][4] = '0'
                                      board[7][0] = '0'
                                      board[7][2] = 'k'
                                      board[7][3] = 'r'
                                      current_score = score(board)
                                    else:
                                        best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                        if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                            checkmate = True
                                            print('hahaha B')
                                        if not checkmate:
                                            board[best_row][best_col] = '0'
                                            board[target_row][target_col] = best_piece
                                            current_score = score(board)
                                            board[best_row][best_col] = best_piece
                                            board[target_row][target_col] = captured
                                    if move == '0-0':
                                      board[7][4] = 'k'
                                      board[7][6] = '0'
                                      board[7][7] = 'r'
                                      board[7][5] = '0'
                                    elif move == '0-0-0':
                                      board[7][4] = 'k'
                                      board[7][0] = 'r'
                                      board[7][2] = '0'
                                      board[7][3] = '0'
                                    if current_score < previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'B')]
                                        if best_piece != 'p':
                                          if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                            analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                          else:
                                            analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                        else:
                                            if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                              analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                            else:
                                              analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'B'))
                                    if checkmate:
                                        return 2, 2, 2, 2, 2, 2
                                board[row][col] = 'B'
                                board[new_row][new_col] = captured_piece
                                checkmate = False
                            else:
                                break
                            if board[new_row][new_col] == 'p' or board[new_row][new_col] == 'n' or board[new_row][new_col] == 'b' or board[new_row][new_col] == 'r' or board[new_row][new_col] == 'q':
                                break
                        else:
                            break

            elif piece == 'R':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'p', 'n', 'b', 'r', 'q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'R'
                                if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                    move = best_move2(board)
                                    if move == '0-0':
                                      board[7][4] = '0'
                                      board[7][6] = 'k'
                                      board[7][7] = '0'
                                      board[7][5] = 'r'
                                      current_score = score(board)
                                    elif move == '0-0-0':
                                      board[7][4] = '0'
                                      board[7][0] = '0'
                                      board[7][2] = 'k'
                                      board[7][3] = 'r'
                                      current_score = score(board)
                                    else:
                                        best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                        if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                            checkmate = True
                                            print('hahaha R')
                                        if not checkmate:
                                            if best_piece != 'p':
                                              if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                                analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                              else:
                                                analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                            else:
                                                if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                                  analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                                else:
                                                  analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                            board[best_row][best_col] = '0'
                                            board[target_row][target_col] = best_piece
                                            current_score = score(board)
                                            board[best_row][best_col] = best_piece
                                            board[target_row][target_col] = captured
                                    if move == '0-0':
                                      board[7][4] = 'k'
                                      board[7][6] = '0'
                                      board[7][7] = 'r'
                                      board[7][5] = '0'
                                    elif move == '0-0-0':
                                      board[7][4] = 'k'
                                      board[7][0] = 'r'
                                      board[7][2] = '0'
                                      board[7][3] = '0'
                                    if current_score < previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'R')]
                                        if best_piece != 'p':
                                          if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                            analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                          else:
                                            analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                        else:
                                            if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                              analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                            else:
                                              analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'R'))
                                    if checkmate:
                                        return 2, 2, 2, 2, 2, 2
                                board[row][col] = 'R'
                                board[new_row][new_col] = captured_piece
                                checkmate = False
                            else:
                                break
                            if board[new_row][new_col] == 'p' or board[new_row][new_col] == 'n' or board[new_row][new_col] == 'b' or board[new_row][new_col] == 'r' or board[new_row][new_col] == 'q':
                                break
                        else:
                            break

            elif piece == 'Q':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'p', 'n', 'b', 'r', 'q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'Q'
                                if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                    move = best_move2(board)
                                    if move == '0-0':
                                      board[7][4] = '0'
                                      board[7][6] = 'k'
                                      board[7][7] = '0'
                                      board[7][5] = 'r'
                                      current_score = score(board)
                                    elif move == '0-0-0':
                                      board[7][4] = '0'
                                      board[7][0] = '0'
                                      board[7][2] = 'k'
                                      board[7][3] = 'r'
                                      current_score = score(board)
                                    else:
                                        best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                        if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                            checkmate = True
                                            print('hahaha Q')
                                        if not checkmate:
                                            board[best_row][best_col] = '0'
                                            board[target_row][target_col] = best_piece
                                            current_score = score(board)
                                            board[best_row][best_col] = best_piece
                                            board[target_row][target_col] = captured
                                    if move == '0-0':
                                      board[7][4] = 'k'
                                      board[7][6] = '0'
                                      board[7][7] = 'r'
                                      board[7][5] = '0'
                                    elif move == '0-0-0':
                                      board[7][4] = 'k'
                                      board[7][0] = 'r'
                                      board[7][2] = '0'
                                      board[7][3] = '0'
                                    if current_score < previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'Q')]
                                        if best_piece != 'p':
                                          if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                            analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                          else:
                                            analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                        else:
                                            if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                              analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                            else:
                                              analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'Q'))
                                    if checkmate:
                                        return 2, 2, 2, 2, 2, 2
                                board[row][col] = 'Q'
                                board[new_row][new_col] = captured_piece
                                checkmate = False
                            else:
                                break
                            if board[new_row][new_col] == 'p' or board[new_row][new_col] == 'n' or board[new_row][new_col] == 'b' or board[new_row][new_col] == 'r' or board[new_row][new_col] == 'q':
                                break
                        else:
                            break

            elif piece == 'K':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1), '0-0', '0-0-0']
                random.shuffle(directions)
                for direction in directions:
                    if direction in {(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)}:
                        new_row = row + direction[0]
                        new_col = col + direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'p', 'n', 'b', 'r', 'q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'K'
                                white_king_row, white_king_col = find_king(board, 'w')
                                if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                    move = best_move2(board)
                                    if move == '0-0':
                                      board[7][4] = '0'
                                      board[7][6] = 'k'
                                      board[7][7] = '0'
                                      board[7][5] = 'r'
                                      current_score = score(board)
                                    elif move == '0-0-0':
                                      board[7][4] = '0'
                                      board[7][0] = '0'
                                      board[7][2] = 'k'
                                      board[7][3] = 'r'
                                      current_score = score(board)
                                    else:
                                        best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                        if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                            checkmate = True
                                            print('hahaha K')
                                        if not checkmate:
                                            board[best_row][best_col] = '0'
                                            board[target_row][target_col] = best_piece
                                            current_score = score(board)
                                            board[best_row][best_col] = best_piece
                                            board[target_row][target_col] = captured
                                    if move == '0-0':
                                      board[7][4] = 'k'
                                      board[7][6] = '0'
                                      board[7][7] = 'r'
                                      board[7][5] = '0'
                                    elif move == '0-0-0':
                                      board[7][4] = 'k'
                                      board[7][0] = 'r'
                                      board[7][2] = '0'
                                      board[7][3] = '0'
                                    if current_score < previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'K')]
                                        if best_piece != 'p':
                                          if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                            analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                          else:
                                            analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                        else:
                                            if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                              analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                            else:
                                              analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'K'))
                                    if checkmate:
                                        return 2, 2, 2, 2, 2, 2
                                board[row][col] = 'K'
                                board[new_row][new_col] = captured_piece
                                checkmate = False
                                white_king_row, white_king_col = find_king(board, 'w')
                    else:
                      if direction == '0-0':
                          if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                              if board[0][4] == 'K' and board[0][7] == 'R' and rook_move_player == '0' and king_move_player == '0' and board[0][5] == '0' and board[0][6] == '0':
                                  board[0][5] = 'K'
                                  board[0][4] = '0'
                                  if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                      board[0][6] = 'K'
                                      board[0][5] = '0'
                                      if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                          board[0][7] = '0'
                                          board[0][5] = 'R'
                                          best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                          if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                              checkmate = True
                                              print('hahaha K')
                                          if not checkmate:
                                              board[best_row][best_col] = '0'
                                              board[target_row][target_col] = best_piece
                                              current_score = score(board)
                                              board[best_row][best_col] = best_piece
                                              board[target_row][target_col] = captured
                                              if current_score < previous_score:
                                                  previous_score = current_score
                                                  best_moves = [('0-0')]
                                                  if best_piece != 'p':
                                                    if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                                      analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                                    else:
                                                      analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                                  else:
                                                      if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                                        analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                                      else:
                                                        analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                              elif current_score == previous_score:
                                                  best_moves.append(('0-0'))
                                          else:
                                              return 2, 2, 2, 2, 2, 2
                                      board[0][4] = 'K'
                                      board[0][6] = '0'
                                      board[0][7] = 'R'
                                      board[0][5] = '0'
                                      checkmate = False
                                      white_king_row, white_king_col = find_king(board, 'w')
                                  else:
                                      board[0][4] = 'K'
                                      board[0][6] = '0'
                              else:
                                  board[0][4] = 'K'
                                  board[0][5] = '0'
                      elif direction == '0-0-0':
                          if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                              if board[0][4] == 'K' and board[0][0] == 'R' and rook_move_player == '0' and king_move_player == '0' and board[0][1] == '0' and board[0][2] == '0' and board[0][3] == '0':
                                  board[0][3] = 'K'
                                  board[0][4] = '0'
                                  if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                      board[0][2] = 'K'
                                      board[0][3] = '0'
                                      if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                          board[0][0] = '0'
                                          board[0][3] = 'R'
                                          best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                          if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                              checkmate = True
                                              print('hahaha K')
                                          if not checkmate:
                                              board[best_row][best_col] = '0'
                                              board[target_row][target_col] = best_piece
                                              current_score = score(board)
                                              board[best_row][best_col] = best_piece
                                              board[target_row][target_col] = captured
                                              if current_score < previous_score:
                                                  previous_score = current_score
                                                  best_moves = [('0-0-0')]
                                                  if best_piece != 'p':
                                                    if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                                      analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                                    else:
                                                      analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                                  else:
                                                      if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                                        analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                                      else:
                                                        analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                              elif current_score == previous_score:
                                                  best_moves.append(('0-0-0'))
                                          else:
                                              return 2, 2, 2, 2, 2, 2
                                      board[0][4] = 'K'
                                      board[0][2] = '0'
                                      board[0][7] = 'R'
                                      board[0][3] = '0'
                                      checkmate = False
                                      white_king_row, white_king_col = find_king(board, 'w')
                                  else:
                                      board[0][4] = 'K'
                                      board[0][2] = '0'
                              else:
                                  board[0][4] = 'K'
                                  board[0][3] = '0'

        if best_moves:
            best_move = random.choice(best_moves)
            if best_move == '0-0':
                board[0][4] = '0'
                board[0][7] = '0'
                board[0][6] = 'k'
                board[0][5] = 'r'
                return '0-0'
            elif best_move == '0-0-0':
                board[0][4] = '0'
                board[0][0] = '0'
                board[0][2] = 'k'
                board[0][3] = 'r'
                return '0-0-0'
            else:
                best_row, best_col, target_row, target_col, best_piece = best_move
                captured_piece = board[target_row][target_col]
                board[best_row][best_col] = '0'
                board[target_row][target_col] = best_piece
              #  if is_king_in_check(board, black_king_row, black_king_col, 'w'):
              #     board[best_row][best_col] = best_piece
                #    board[target_row][target_col] = captured_piece
                return best_row, best_col, target_row, target_col, best_piece, captured_piece
        else:
            return 1, 1, 1, 1, 1, 1

def best_move2(board):
    global king_move
    global rook_move
    black_king_row, black_king_col = find_king(board, 'b')
    previous_score = -60
    best_moves = []
    rows = list(range(8))
    random.shuffle(rows)
    cols = list(range(8))
    random.shuffle(cols)
    for row in rows:
        for col in cols:
            piece = board[row][col]
            if piece == 'p':
                directions = [1, 2, 3, 4]
                random.shuffle(directions)
                for direction in directions:
                    if direction == 1 and row == 1 and board[row-2][col] == '0' and board[row-1][col] == '0':
                        board[row][col] = '0'
                        board[row-2][col] = 'p'
                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                            current_score = score(board)
                            if current_score > previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row-2, col, 'p')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row-2, col, 'p'))
                        board[row][col] = 'p'
                        board[row-2][col] = '0'

                    elif direction == 2 and row < 7 and board[row-1][col] == '0':
                        board[row][col] = '0'
                        board[row-1][col] = 'p'
                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                            current_score = score(board)
                            if current_score > previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row-1, col, 'p')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row-1, col, 'p'))
                        board[row][col] = 'p'
                        board[row-1][col] = '0'

                    elif direction == 3 and row < 7 and col > 0 and board[row-1][col-1] in {'P', 'N', 'B', 'R', 'Q'}:
                        captured_piece = board[row-1][col-1]
                        board[row][col] = '0'
                        board[row-1][col-1] = 'p'
                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                            current_score = score(board)
                            if current_score > previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row-1, col-1, 'p')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row-1, col-1, 'p'))
                        board[row][col] = 'p'
                        board[row-1][col-1] = captured_piece

                    elif direction == 4 and row > 0 and col < 7 and board[row-1][col+1] in {'P', 'N', 'B', 'R', 'Q'}:
                        captured_piece = board[row-1][col+1]
                        board[row][col] = '0'
                        board[row-1][col+1] = 'p'
                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                            current_score = score(board)
                            if current_score > previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row-1, col+1, 'p')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row-1, col+1, 'p'))
                        board[row][col] = 'p'
                        board[row-1][col+1] = captured_piece

            elif piece == 'n':
                directions = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]
                random.shuffle(directions)
                for direction in directions:
                    new_row = row + direction[0]
                    new_col = col + direction[1]
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                            captured_piece = board[new_row][new_col]
                            board[row][col] = '0'
                            board[new_row][new_col] = 'n'
                            if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                current_score = score(board)
                                if current_score > previous_score:
                                    previous_score = current_score
                                    best_moves = [(row, col, new_row, new_col, 'n')]
                                elif current_score == previous_score:
                                    best_moves.append((row, col, new_row, new_col, 'n'))
                            board[row][col] = 'n'
                            board[new_row][new_col] = captured_piece

            elif piece == 'b':
                directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'b'
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    current_score = score(board)
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'b')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'b'))
                                board[row][col] = 'b'
                                board[new_row][new_col] = captured_piece
                            else:
                                break
                            if board[new_row][new_col] == 'P' or board[new_row][new_col] == 'N' or board[new_row][new_col] == 'B' or board[new_row][new_col] == 'R' or board[new_row][new_col] == 'Q':
                                break
                        else:
                            break

            elif piece == 'r':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'r'
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    current_score = score(board)
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'r')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'r'))
                                board[row][col] = 'r'
                                board[new_row][new_col] = captured_piece
                            else:
                                break
                            if board[new_row][new_col] == 'P' or board[new_row][new_col] == 'N' or board[new_row][new_col] == 'B' or board[new_row][new_col] == 'R' or board[new_row][new_col] == 'Q':
                                break
                        else:
                            break

            elif piece == 'q':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'q'
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    current_score = score(board)
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'q')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'q'))
                                board[row][col] = 'q'
                                board[new_row][new_col] = captured_piece
                            else:
                                break
                            if board[new_row][new_col] == 'P' or board[new_row][new_col] == 'N' or board[new_row][new_col] == 'B' or board[new_row][new_col] == 'R' or board[new_row][new_col] == 'Q':
                                break
                        else:
                            break

            elif piece == 'k':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1), '0-0', '0-0-0']
                random.shuffle(directions)
                for direction in directions:
                    if direction in {(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)}:
                        new_row = row + direction[0]
                        new_col = col + direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'k'
                                black_king_row, black_king_col = find_king(board, 'b')
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    current_score = score(board)
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'k')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'k'))
                                board[row][col] = 'k'
                                board[new_row][new_col] = captured_piece
                                black_king_row, black_king_col = find_king(board, 'b')
                    else:
                      if direction == '0-0':
                          if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                              if board[7][4] == 'k' and board[7][7] == 'r' and rook_move == '0' and king_move == '0' and board[7][5] == '0' and board[7][6] == '0':
                                  board[7][5] = 'k'
                                  board[7][4] = '0'
                                  if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                      board[7][6] = 'k'
                                      board[7][5] = '0'
                                      if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                          board[7][7] = '0'
                                          board[7][5] = 'r'
                                          current_score = score(board)
                                          if current_score > previous_score:
                                              previous_score = current_score
                                              best_moves = [('0-0')]
                                          elif current_score == previous_score:
                                              best_moves.append(('0-0'))
                                          board[7][4] = 'k'
                                          board[7][6] = '0'
                                          board[7][7] = 'r'
                                          board[7][5] = '0'
                                          black_king_row, black_king_col = find_king(board, 'b')
                                  else:
                                      board[7][4] = 'k'
                                      board[7][6] = '0'
                              else:
                                  board[7][4] = 'k'
                                  board[7][5] = '0'
                      elif direction == '0-0-0':
                          if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                              if board[7][4] == 'k' and board[7][0] == 'r' and rook_move == '0' and king_move == '0' and board[7][1] == '0' and board[7][2] == '0' and board[7][3] == '0':
                                  board[7][3] = 'k'
                                  board[7][4] = '0'
                                  if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                      board[7][2] = 'k'
                                      board[7][3] = '0'
                                      if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                          board[7][0] = '0'
                                          board[7][3] = 'r'
                                          current_score = score(board)
                                          if current_score > previous_score:
                                              previous_score = current_score
                                              best_moves = [('0-0-0')]
                                          elif current_score == previous_score:
                                              best_moves.append(('0-0-0'))
                                          board[7][4] = 'k'
                                          board[7][2] = '0'
                                          board[7][0] = 'r'
                                          board[7][3] = '0'
                                          black_king_row, black_king_col = find_king(board, 'b')
                                  else:
                                      board[7][4] = 'k'
                                      board[7][2] = '0'
                              else:
                                  board[7][4] = 'k'
                                  board[7][3] = '0'

    if best_moves:
        best_move = random.choice(best_moves)
        if best_move != '0-0' and best_move != '0-0-0':
            best_row2, best_col2, target_row2, target_col2, best_piece2 = best_move
            captured_piece2 = board[target_row2][target_col2]
            board[best_row2][best_col2] = '0'
            board[target_row2][target_col2] = best_piece2
            return best_row2, best_col2, target_row2, target_col2, best_piece2, captured_piece2
        elif best_move == '0-0':
            king_move = '1'
            board[7][4] = '0'
            board[7][7] = '0'
            board[7][6] = 'k'
            board[7][5] = 'r'
            return '0-0-0'
        elif best_move == '0-0-0':
            king_move = '1'
            board[7][4] = '0'
            board[7][0] = '0'
            board[7][2] = 'k'
            board[7][3] = 'r'
            return '0-0-0'
        else:
            return 1, 1, 1, 1, 1, 1
def game_loop():
  for _ in range(100):
    white_king_row, white_king_col = find_king(board, 'w')
    black_king_row, black_king_col = find_king(board, 'b')
    if is_king_in_check(board, white_king_row, white_king_col, 'w'):
      print("White king is in check!")
      players_turn(board)
    elif is_king_in_check(board, black_king_row, black_king_col, 'b'):
      print("Black king is in check!")
      best_move(board)
    else:
      best_move(board)

game_loop()

Kd1
g5
Ng5
-10.999999999999996

n N B Q K B N R
P P 0 P P P 0 P
0 0 0 0 0 0 0 0
0 0 0 0 0 0 n 0
0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0
p p p p p p p p
r 0 b 0 k 0 0 r

In check
n N B Q K B N R
P P 0 P P P P P
0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0
0 0 0 0 0 n 0 0
p p p p p p p p
r 0 b 0 0 r k 0
CHECKMATE! you lose
Rb1
Nd4
17.699999999999957

0 0 B 0 K 0 N R
P P 0 P P P P P
n 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0
0 0 0 n 0 0 0 0
0 0 0 0 0 0 0 0
p p p p p p p p
0 r b 0 k 0 0 r

0 0 B 0 K 0 N R
P 1 0 P P P P P
n 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0
0 0 0 0 0 n 0 0
p p p p p p p p
0 r b 0 k 0 0 r
CHECKMATE! you lose
Rg1


AttributeError: 'int' object has no attribute 'lower'

h3
Re1
Ke1
-5.400000000000046

R 0 0 0 0 0 0 0
0 P P 0 0 K P 0
P 0 P 0 0 0 0 N
0 0 0 0 0 B P P
0 B p p 0 p 0 0
0 0 0 0 0 0 0 p
p p 0 n 0 0 p 0
r 0 b 0 k 0 0 0

Nb1
hahaha R
Nb3
Ne4
Nf3
Nf1
b3
g3
g4
a4
a3
Ke2
In check
Kc2
In check
Kxe1
Rb1
R 0 0 0 0 0 0 0
0 P P 0 0 K P 0
P 0 P 0 0 0 0 N
0 0 0 0 0 B p P
0 B p p 0 0 0 0
0 0 0 0 0 0 0 0
p p 0 n 0 0 p p
r 0 b k R 0 0 0

g5
-3.4000000000000457